In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.cross_validation import cross_val_score
from keras import backend as K

df = pd.read_pickle("./trainingSet")
df = df.drop(columns=['DMA'])

dfTrain = pd.read_csv("./NewTrainSet")
dfTrain = dfTrain.drop(columns=['DMA', 'Unnamed: 0'])

dfTest = pd.read_csv("./NewTestSet")
dfTest = dfTest.drop(columns=['DMA', 'Unnamed: 0'])

dfTrain.head()

,Flow,dev,t-4dev,t-8dev,t-12dev,t-16dev,t-20dev,t-24dev,t-28dev,t-32dev,t-36dev,t-40dev,t20Leak,t24Leak,t28Leak,t32Leak,t36Leak,t40Leak,LeakFixIn4Days
0,0.089,0.026370,0.014608,-0.020870,-0.018630,0.014765,0.031275,0.015370,0.003608,-0.010922,-0.005509,-0.032870,False,False,False,False,False,False,False
1,0.033,-0.029000,0.029451,-0.012333,-0.018000,0.023451,0.047627,0.027000,0.018451,-0.010098,-0.005000,-0.001333,False,False,False,False,False,False,False
2,0.067,-0.006370,0.097098,-0.053370,0.048630,-0.009314,-0.015118,0.093630,-0.013902,0.034902,-0.024925,0.002630,False,False,False,False,False,False,False
3,0.111,0.039222,0.036569,-0.003815,-0.004778,0.029627,-0.023275,0.128222,0.025569,-0.009941,0.023528,0.018185,False,False,False,False,False,False,False
4,0.078,0.014608,-0.020870,-0.018630,0.014765,0.031275,0.015370,0.003608,-0.010922,-0.005509,-0.032870,-0.010922,False,False,False,False,False,False,False


In [18]:
def df_to_numpy_array(df):
    num_columns = list(df.select_dtypes(include=['float64', 'int64']).columns.values)
    num_values = df[num_columns].values
    bool_columns = list(df.select_dtypes(include=['bool']).columns.values)
    bool_values = df[bool_columns].values
    return np.concatenate((bool_values,num_values),axis=1)

In [19]:
def model():
    model = Sequential()
    model.add(Dense(12, input_dim=12, init='normal', activation='relu'))
    model.add(Dense(12, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
dfTrain.head()

,Flow,dev,t-4dev,t-8dev,t-12dev,t-16dev,t-20dev,t-24dev,t-28dev,t-32dev,t-36dev,t-40dev,t20Leak,t24Leak,t28Leak,t32Leak,t36Leak,t40Leak,LeakFixIn4Days
0,0.089,0.026370,0.014608,-0.020870,-0.018630,0.014765,0.031275,0.015370,0.003608,-0.010922,-0.005509,-0.032870,False,False,False,False,False,False,False
1,0.033,-0.029000,0.029451,-0.012333,-0.018000,0.023451,0.047627,0.027000,0.018451,-0.010098,-0.005000,-0.001333,False,False,False,False,False,False,False
2,0.067,-0.006370,0.097098,-0.053370,0.048630,-0.009314,-0.015118,0.093630,-0.013902,0.034902,-0.024925,0.002630,False,False,False,False,False,False,False
3,0.111,0.039222,0.036569,-0.003815,-0.004778,0.029627,-0.023275,0.128222,0.025569,-0.009941,0.023528,0.018185,False,False,False,False,False,False,False
4,0.078,0.014608,-0.020870,-0.018630,0.014765,0.031275,0.015370,0.003608,-0.010922,-0.005509,-0.032870,-0.010922,False,False,False,False,False,False,False


In [27]:
crossValidationX = df_to_numpy_array(df.drop(columns=["LeakFixIn4Days", "t20Leak", "t24Leak", "t28Leak", "t32Leak", "t36Leak", "t40Leak"]))
crossValidationY = df_to_numpy_array(df.drop(columns=["Flow","dev","t-4dev","t-8dev","t-12dev","t-16dev","t-20dev","t-24dev","t-28dev","t-32dev","t-36dev","t-40dev","t20Leak","t24Leak","t28Leak","t32Leak","t36Leak","t40Leak"]))
XTrain = df_to_numpy_array(dfTrain.drop(columns=["LeakFixIn4Days", "t20Leak", "t24Leak", "t28Leak", "t32Leak", "t36Leak", "t40Leak"]))
XTest = df_to_numpy_array(dfTest.drop(columns=["LeakFixIn4Days", "t20Leak", "t24Leak", "t28Leak", "t32Leak", "t36Leak", "t40Leak"]))
yTrain = df_to_numpy_array(dfTrain.drop(columns=["Flow","dev","t-4dev","t-8dev","t-12dev","t-16dev","t-20dev","t-24dev","t-28dev","t-32dev","t-36dev","t-40dev","t20Leak", "t24Leak", "t28Leak", "t32Leak", "t36Leak", "t40Leak"]))
yTest = df_to_numpy_array(dfTest.drop(columns=["Flow","dev","t-4dev","t-8dev","t-12dev","t-16dev","t-20dev","t-24dev","t-28dev","t-32dev","t-36dev","t-40dev","t20Leak","t24Leak","t28Leak","t32Leak","t36Leak","t40Leak"]))

In [28]:
crossValidationX[0]

array([ 0.089     ,  0.02637037,  0.01460784, -0.02087037, -0.01862963,
        0.01476471,  0.03127451,  0.01537037,  0.00360784, -0.01092157,
       -0.00550943, -0.03287037])

In [29]:
kr = KerasRegressor(build_fn=model, nb_epoch=1, batch_size=5, verbose=1)

scores = cross_val_score(kr, crossValidationX, crossValidationY, cv=10)
print("logloss = %4.2f std = %4.2f" % (scores.mean(),scores.std()))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=12, activation="relu", kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, activation="relu", kernel_initializer="normal")`
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="normal")`
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`
  


Epoch 1/1
73912/73912 [==============================] - 5s 65us/step
Epoch 1/1
73911/73911 [==============================] - 5s 68us/step
Epoch 1/1
392435/665200 [================>.............] - ETA: 35s - loss: 0.2629 - acc: 0.9244

KeyboardInterrupt: 

In [32]:
    model = Sequential()
    model.add(Dense(12, input_dim=12, init='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(6, init='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(6, init='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(XTrain, yTrain, validation_data=(XTest,yTest), epochs=20, batch_size=5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=12, activation="relu", kernel_initializer="normal")`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="normal")`
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(6, activation="relu", kernel_initializer="normal")`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`
  


Train on 100000 samples, validate on 12520 samples
Epoch 1/20
100000/100000 [==============================] - 16s 163us/step - loss: 0.6140 - acc: 0.6807 - val_loss: 0.4921 - val_acc: 0.8615
Epoch 2/20
100000/100000 [==============================] - 16s 160us/step - loss: 0.6043 - acc: 0.6933 - val_loss: 0.5083 - val_acc: 0.8431
Epoch 3/20
100000/100000 [==============================] - 16s 158us/step - loss: 0.6027 - acc: 0.6949 - val_loss: 0.4981 - val_acc: 0.8388
Epoch 4/20
100000/100000 [==============================] - 16s 157us/step - loss: 0.6024 - acc: 0.6958 - val_loss: 0.5381 - val_acc: 0.8024
Epoch 5/20
100000/100000 [==============================] - 16s 159us/step - loss: 0.6006 - acc: 0.6972 - val_loss: 0.5055 - val_acc: 0.8419
Epoch 6/20
100000/100000 [==============================] - 16s 159us/step - loss: 0.6011 - acc: 0.6967 - val_loss: 0.5250 - val_acc: 0.7997
Epoch 7/20
100000/100000 [==============================] - 16s 158us/step - loss: 0.6005 - acc: 0.6974